In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import gc

In [ ]:
df = pd.read_pickle('../data/reddit_corpus_balanced_filtered.gzip', compression='gzip')

In [ ]:
# Creating dictionary for theme groups
theme_groups = {}
theme_groups['SARS-CoV-2'] = ['r/CovidVaccinated', 'r/Masks4All', 'r/NoLockdownsNoMasks', 'r/EndTheLockdowns', 'r/COVID19', 'r/COVID19positive', 'r/CoronavirusCanada', 'r/CoronavirusRecession', 'r/CoronavirusUK', 'r/CoronavirusUS', 'r/Coronavirus', 'r/LockdownSkepticism']
theme_groups['Vaccines'] = ['r/CovidVaccinated', 'r/VACCINES', 'r/vaxxhappened', 'r/AntiVaxxers', 'r/antivax', 'r/TrueAntiVaccination', 'r/DebateVaccine', 'r/DebateVaccines']
theme_groups['Abortion'] = ['r/AskProchoice', 'r/prochoice', 'r/insaneprolife', 'r/prolife', 'r/ProLifeLibertarians', 'r/Abortiondebate', 'r/abortion']
theme_groups['womens-and-mens-rights'] = ['r/Feminism', 'r/feminisms', 'r/RadicalFeminism', 'r/RadicalFeminismUSA', 'r/MRActivism', 'r/MensRights', 'r/antifeminists', 'r/feminismformen', 'r/masculism', 'r/GenderCritical', 'r/Egalitarianism']
theme_groups['Gun-control'] = ['r/Firearms', 'r/GunsAreCool', 'r/liberalgunowners', 'r/progun', 'r/guncontrol', 'r/GunDebates', 'r/GunResearch', 'r/gunpolitics']
theme_groups['Climate-change'] = ['r/climateskeptics', 'r/GlobalClimateChange', 'r/climate', 'r/climatechange']
theme_groups['5G'] = ['r/5GDebate']
theme_groups['general-political-debate'] = ['r/JoeBiden', 'r/LeftistsForMen', 'r/Liberal', 'r/LockdownCriticalLeft', 'r/democrats', 'r/Conservative', 'r/ConservativesOnly', 'r/conservatives', 'r/Republican', 'r/RepublicanValues', 'r/politics', 'r/uspolitics']

# !ProLifeLibertarians is missing any labled data

#theme_groups

In [ ]:
# Extract data end sort to dictionaries for later plots
sub_reddit_dic = {}

for row in df.iterrows():
    data = row[1]
    documents = data['documents']
    
    for doc_id, text, date, sub_reddit, labels in documents:
        if len(labels) > 0:
            #print(labels)
            if sub_reddit not in sub_reddit_dic:
                sub_reddit_dic[sub_reddit] = {'politics': {'count':0,'LEFT_CENTER':0,'LEFT':0,'LEAST_BIASED':0,'RIGHT_CENTER':0,'SATIRE':0,'PRO_SCIENCE':0,'RIGHT':0,'EXTREME_LEFT':0,'CONSPIRACY_PSEUDOSCIENCE':0,'EXTREME_RIGHT':0,'PRO_RUSSIAN_PROPAGANDA':0}, 'news': {'count':0,'HIGH':0,'VERY_HIGH':0,'MOSTLY_FACTUAL':0,'MIXED':0,'VERY_LOW':0,'LOW':0}}
            for label in labels:
                pol_bias = label[2][0]
                news_bias = label[3]
                if len(pol_bias) > 0:
                    sub_reddit_dic[sub_reddit]['politics'][pol_bias] += 1
                    sub_reddit_dic[sub_reddit]['politics']['count'] += 1
                if (len(news_bias) > 0):
                    sub_reddit_dic[sub_reddit]['news'][news_bias] += 1
                    sub_reddit_dic[sub_reddit]['news']['count'] += 1
                    
theme_dic = {}

for theme in theme_groups:
    theme_dic[theme] = {'politics': {'count':0,'LEFT_CENTER':0,'LEFT':0,'LEAST_BIASED':0,'RIGHT_CENTER':0,'SATIRE':0,'PRO_SCIENCE':0,'RIGHT':0,'EXTREME_LEFT':0,'CONSPIRACY_PSEUDOSCIENCE':0,'EXTREME_RIGHT':0,'PRO_RUSSIAN_PROPAGANDA':0}, 'news': {'count':0,'HIGH':0,'VERY_HIGH':0,'MOSTLY_FACTUAL':0,'MIXED':0,'VERY_LOW':0,'LOW':0}}
    for sub_reddit in theme_groups[theme]:
        if sub_reddit not in sub_reddit_dic:
            continue
        for i in sub_reddit_dic[sub_reddit]['politics']:
            theme_dic[theme]['politics'][i] += sub_reddit_dic[sub_reddit]['politics'][i]
        for i in sub_reddit_dic[sub_reddit]['news']:
            theme_dic[theme]['news'][i] += sub_reddit_dic[sub_reddit]['news'][i]

In [ ]:
del df # Free up some RAM
gc.collect();

In [ ]:
# Plot of news distribution in categories. Singel Plot
plt.figure(figsize=(16,8))
cmap = ['orangered', 'lime', 'aqua', 'violet', 'gold', 'grey', 'blue', 'darkmagenta']
categories = ['VERY_HIGH','HIGH','MOSTLY_FACTUAL','MIXED','LOW','VERY_LOW']
width = 1/len(theme_dic)
max_y = 0
for ind, theme in enumerate(theme_dic):
    data = theme_dic[theme]['news']
    n = data['count']
    heights = [data[i]/n if i in data else 0 for i in categories]
    if(max(heights) > max_y):
            max_y = max(heights)
    plt.bar(x = [i*3+ind*(1.5*width)-(1.5*width*(len(theme_dic)-1)/2) for i in range(len(categories))], height=heights, width=width, color=cmap[ind])
plt.xticks([i*3 for i in range(len(categories))],categories)
plt.legend(theme_dic)
plt.ylabel("relative percentage")
plt.yticks([i/10 for i in range(min(11,int(max_y*10) + 2))], [str(i*10)+'%' for i in range(min(11,int(max_y*10) + 2))])
plt.title("News bias in all groups")
plt.savefig("./overview/news.pdf")
plt.show()

# Plot of political distribution in categories. Singel Plot
plt.figure(figsize=(16,8))
cmap = ['orangered', 'lime', 'aqua', 'violet', 'gold', 'grey', 'blue', 'darkmagenta']
categories = ['EXTREME_LEFT','LEFT','LEFT_CENTER','LEAST_BIASED','RIGHT_CENTER','RIGHT','EXTREME_RIGHT','CONSPIRACY_PSEUDOSCIENCE','PRO_RUSSIAN_PROPAGANDA', 'PRO_SCIENCE']
max_y = 0
for ind, theme in enumerate(theme_dic):
    data = theme_dic[theme]['politics']
    n = data['count']
    heights = [data[i]/n if i in data else 0 for i in categories]
    if(max(heights) > max_y):
            max_y = max(heights)
    plt.bar(x = [i*3+ind*(1.5*width)-(1.5*width*(len(theme_dic)-1)/2) for i in range(len(categories))], height=heights, width=width, color=cmap[ind])
plt.xticks([i*3 for i in range(len(categories))],[j if ind % 2 == 0 else " \n"+j for ind,j in enumerate(categories)])
plt.legend(theme_dic)
plt.ylabel("relative percentage")
plt.yticks([i/10 for i in range(min(11,int(max_y*10) + 2))], [str(i*10)+'%' for i in range(min(11,int(max_y*10) + 2))])
plt.title("Political bias in all groups")
plt.savefig("./overview/political.pdf")
plt.show()

In [ ]:
# Each subreddit of a category

# custom colormap. len = 12, since the biggest category consists out of 12 sub-reddits
cmap = ['orangered', 'darkmagenta', 'aqua', 'violet', 'gold', 'grey', 'blue', 'lime', 'steelblue', 'silver', 'deeppink', 'olivedrab']

# POLITICAL BIAS
# Ignore certian categories
categories = ['EXTREME_LEFT','LEFT','LEFT_CENTER','LEAST_BIASED','RIGHT_CENTER','RIGHT','EXTREME_RIGHT']#,'CONSPIRACY_PSEUDOSCIENCE','PRO_RUSSIAN_PROPAGANDA','PRO_SCIENCE']
for group in theme_groups:
    plt.figure(figsize=(16,8))
    max_y = 0
    
    sub_reddits_in_group = [i for i in theme_groups[group] if i in sub_reddit_dic]
    for ind, sub_reddit in enumerate(sub_reddits_in_group):
        
        data = sub_reddit_dic[sub_reddit]['politics']
        # normalization
        n = data['count'] - data['CONSPIRACY_PSEUDOSCIENCE'] - data['PRO_RUSSIAN_PROPAGANDA'] - data['PRO_SCIENCE']
        
        width = 1/len(sub_reddits_in_group)
        heights = [data[i]/n if i in data else 0 for i in categories]
        
        # max hight for y-axis labels
        if(max(heights) > max_y):
            max_y = max(heights)
        
        plt.bar(x = [i*3+ind*(1.5*width)-(1.5*width*(len(sub_reddits_in_group)-1)/2) for i in range(len(categories))], height=heights, width=width, color=cmap[ind])
    plt.xticks([i*3 for i in range(len(categories))],[j.replace('_', ' ') for j in categories])
    plt.legend(sub_reddits_in_group)
    plt.title('Political tendencies for sub-reddits in group: ' + group, fontsize=20)
    plt.ylabel('relative percentage', fontsize=16)
    plt.yticks([i/10 for i in range(min(11,int(max_y*10) + 2))], [str(i*10)+'%' for i in range(min(11,int(max_y*10) + 2))])
    plt.savefig('./categories/' + group + '_politics.pdf')
    plt.show()

# FACTUAL STRUCTURE
categories = ['VERY_HIGH','HIGH','MOSTLY_FACTUAL','MIXED','LOW','VERY_LOW']
for group in theme_groups:
    plt.figure(figsize=(16,8))
    max_y = 0
    # Dont iterate over sub-reddits with no labeled data
    sub_reddits_in_group = [i for i in theme_groups[group] if i in sub_reddit_dic]
    for ind, sub_reddit in enumerate(sub_reddits_in_group):
        if (sub_reddit not in sub_reddit_dic):
            continue
        data = sub_reddit_dic[sub_reddit]['news']
        n = data['count']
        width = 1/len(sub_reddits_in_group)
        heights = [data[i]/n if i in data else 0 for i in categories]
        if(max(heights) > max_y):
            max_y = max(heights)
        plt.bar(x = [i*3+ind*(1.5*width)-(1.5*width*(len(sub_reddits_in_group)-1)/2) for i in range(len(categories))], height=heights, width=width, color=cmap[ind])
    plt.xticks([i*3 for i in range(len(categories))],[j.replace('_', ' ') for j in categories])
    plt.legend(sub_reddits_in_group)
    plt.title('News tendencies for sub-reddits in group: ' + group, fontsize=20)
    plt.ylabel('relative percentage', fontsize=16)
    plt.yticks([i/10 for i in range(min(11,int(max_y*10) + 2))], [str(i*10)+'%' for i in range(min(11,int(max_y*10) + 2))])
    plt.savefig('./categories/' + group + '_news.pdf')
    plt.show()

In [ ]:
all_post_dic = {}

for row in df.iterrows():
    data = row[1]
    documents = data['documents']
    
    for doc_id, text, date, sub_reddit, labels in documents:
        if sub_reddit not in all_post_dic:
            all_post_dic[sub_reddit] = {'total_posts': 0, 'real_news': 0, 'fake_news': 0}
        all_post_dic[sub_reddit]['total_posts'] += 1
        if len(labels) > 0:
            fake = False
            
            # If any fakenews in post mark post as fake
            # If counting everything: 'real_news': 102024, 'fake_news': 10557
            # If any label is fake_news lable as fake_news: 'real_news': 69182, 'fake_news': 8884
            for page, fake_news, political_bias, factual_bias in labels:
                if fake_news == 1:
                    fake = True
                    break
            
            if fake:
                all_post_dic[sub_reddit]['fake_news'] += 1
            else:
                all_post_dic[sub_reddit]['real_news'] += 1

all_post_dic['TOTAL'] = {'total_posts': 0, 'real_news': 0, 'fake_news': 0}
for theme in theme_groups:
    data = theme_groups[theme]
    all_post_dic[theme] = {'total_posts': 0, 'real_news': 0, 'fake_news': 0}
    for sub_reddit in data:
        all_post_dic[theme]['total_posts'] += all_post_dic[sub_reddit]['total_posts']
        all_post_dic[theme]['real_news'] += all_post_dic[sub_reddit]['real_news']
        all_post_dic[theme]['fake_news'] += all_post_dic[sub_reddit]['fake_news']
    all_post_dic['TOTAL']['total_posts'] += all_post_dic[theme]['total_posts']
    all_post_dic['TOTAL']['real_news'] += all_post_dic[theme]['real_news']
    all_post_dic['TOTAL']['fake_news'] += all_post_dic[theme]['fake_news']

In [ ]:
def disable_box(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)

def make_dist_plot(group, subs):
    data = all_post_dic[group]
    fig, ax = plt.subplots(5, 1, figsize=(10,8))
    plt.subplots_adjust(hspace=1)
    n = data['total_posts']
    rn = data['real_news']
    fn = data['fake_news']

    for a in ax:
        disable_box(a)
    ax[0].set_title('Overall distribution of labeled and unlabeled posts')
    ax[0].barh(0,(rn+fn)/n, color='red', left=0)
    ax[0].barh(0,(n-(rn+fn))/n, color='steelblue', left=(rn+fn)/n)
    ax[0].set_xticks([0.5*(rn+fn)/n, 0.5*(n-(rn+fn))/n + (rn+fn)/n],['Labeled Posts - {:.2f}%'.format(100*(rn+fn)/n), 'Unlabeled Posts - {:.2f}%'.format(100*(n-(rn+fn))/n)])
    ax[0].set_yticks([]);

    labeled_n = rn+fn
    ax[1].set_title('Distribution of real and fake news in labeled posts')
    ax[1].barh(0,rn/labeled_n, color='orange', left=0)
    ax[1].barh(0,fn/labeled_n, color='indianred', left=rn/labeled_n)
    ax[1].set_xticks([0.5*rn/labeled_n, 0.5*fn/labeled_n + rn/labeled_n],['Real News Posts - {:.2f}%'.format(100*rn/labeled_n), 'Fake News Posts - {:.2f}%'.format(100*fn/labeled_n)])
    ax[1].set_yticks([]);

    # custom colormap. len = 12, since the biggest category consists out of 12 sub-reddits
    cmap = ['orangered', 'darkmagenta', 'aqua', 'violet', 'gold', 'grey', 'blue', 'lime', 'steelblue', 'silver', 'deeppink', 'olivedrab']
    
    cur_n = 0
    ax[2].set_title('Distribution of posts in groups')
    l = []
    for ind, sub in enumerate(subs):
        ax[2].barh(0, all_post_dic[sub]['total_posts']/n, left=cur_n, color=cmap[ind])
        cur_n += (all_post_dic[sub]['total_posts']/n)
    ax[2].set_xticks([])
    ax[2].set_yticks([])
    
    cur_rn = 0
    ax[3].set_title('Distribution of real news in groups')
    l = []
    for ind, sub in enumerate(subs):
        l.append(ax[3].barh(0, all_post_dic[sub]['real_news']/rn, left=cur_rn, color=cmap[ind])[0])
        cur_rn += (all_post_dic[sub]['real_news']/rn)
    ax[3].set_xticks([])
    ax[3].set_yticks([])
    plt.legend(l,subs,bbox_to_anchor=(1.28, 5.07))

    cur_fn = 0
    ax[4].set_title('Distribution of fake news in groups')
    l = []
    for ind, sub in enumerate(subs):
        l.append(ax[4].barh(0, all_post_dic[sub]['fake_news']/fn, left=cur_fn, color=cmap[ind])[0])
        cur_fn += (all_post_dic[sub]['fake_news']/fn)
    ax[4].set_xticks([])
    ax[4].set_yticks([]);
    
    plt.savefig('./categories/' + group + '_dist.pdf', bbox_inches='tight')
    plt.show();

In [ ]:
for group in theme_groups:
    subs = theme_groups[group]
    make_dist_plot(group, subs)